In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")

In [4]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [5]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep = "\n",
                                   index= False,
                                   header= False
                                   )

In [6]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 0
Created a chunk of size 1214, which is longer than the specified 0
Created a chunk of size 373, which is longer than the specified 0
Created a chunk of size 309, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 482, which is longer than the specified 0
Created a chunk of size 960, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 843, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1088, which is longer than the specified 0
Created a chunk of size 1189, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Create

In [7]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [13]:
import os
# Check if API key is loaded correctly
google_api_key = os.getenv("GEMINI_API_KEY")
if not google_api_key:
    raise ValueError("GEMINI_API_KEY not found. Make sure it is set in your .env file.")


# Initialize Gemini embeddings
embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)


db_books = Chroma.from_documents(
    documents,
    embedding= embedding_model
)

In [14]:
query = "A book about a women in roman empire."
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='e51ebf4e-5d9c-4294-9ceb-c403978b9c53', metadata={'source': 'tagged_description.txt'}, page_content='9780380710843 The fourth novel of the Masters of Rome series focuses on the women in the life of the Roman emperor Gaius Julius Caesar at the height of his power'),
 Document(id='d7d20bb9-5690-457d-88ab-e38f29a0ee6f', metadata={'source': 'tagged_description.txt'}, page_content='9780679724773 The emperor Claudius tells of his life during the reigns of Augustus, Tiberius, and Caligula and the events that led to his rise to power in a classic novel reconstructing ancient Rome'),
 Document(id='e47a4317-aa4c-4e55-ae79-acf597743cd9', metadata={'source': 'tagged_description.txt'}, page_content='"9780374253844 Floria Aemilia, a concubine to St. Augustine for more than ten years prior to his spiritual awakening, challenges St. Augustine\'s view of women and love as described in his ""Confessions,"" in a fictional letter addressing faith, doubt, and human relationships"'),
 Document(

In [15]:
# Find the match of the first text given by the query and the isbn13 number of the dataframe 
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
1754,9780380710843,0380710846,Caesar's Women,Colleen McCullough,Fiction,http://books.google.com/books/content?id=yFK8F...,The fourth novel of the Masters of Rome series...,1997.0,4.24,943.0,5277.0,Caesar's Women,9780380710843 The fourth novel of the Masters ...


In [16]:
def retrieve_semantic_recommendations(query: str) -> pd.DataFrame:
    """
    Retrieve top-k semantic recommendations based on a query.

    Args:
        query (str): The search query to find relevant books.
        top_k (int): The number of top recommendations to return. Default is 10.

    Returns:
        pd.DataFrame: A DataFrame containing the top-k recommended books.
    """
    # Perform a similarity search on the database with the given query
    recs = db_books.similarity_search(query, k=10)
    
    # Extract the ISBN13 numbers from the search results
    books_list = [int(rec.page_content.strip('"').split()[0]) for rec in recs]
    
    # Filter the books DataFrame to include only the top-k recommendations
    return books[books["isbn13"].isin(books_list)]

In [17]:
retrieve_semantic_recommendations("A book about a women in roman empire.")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
1223,9780312320195,0312320191,SPQR VI: Nobody Loves a Centurion,John Maddox Roberts,Fiction,http://books.google.com/books/content?id=Wkowm...,"Like so many young men in later generations, R...",2003.0,4.13,288.0,626.0,SPQR VI: Nobody Loves a Centurion,9780312320195 Like so many young men in later ...
1242,9780312357771,031235777X,The Silver Pigs,Lindsey Davis,Fiction,http://books.google.com/books/content?id=a0wmu...,"When Marcus Didius Falco, a Roman ""informer"" w...",2006.0,3.94,329.0,8592.0,The Silver Pigs: A Marcus Didius Falco Mystery,"9780312357771 When Marcus Didius Falco, a Roma..."
1622,9780373802135,0373802137,Elphame's Choice,P. C. Cast,Fiction,http://books.google.com/books/content?id=dp0NA...,"This follow-up to ""Goddess by Mistake"" returns...",2004.0,4.02,551.0,3894.0,Elphame's Choice,"9780373802135 This follow-up to ""Goddess by Mi..."
1634,9780374253844,0374253846,That Same Flower,Jostein Gaarder;Anne Born,Fiction,http://books.google.com/books/content?id=kgVmQ...,"Floria Aemilia, a concubine to St. Augustine f...",1998.0,3.67,167.0,68.0,That Same Flower: Floria Aemilia's Letter to S...,"9780374253844 Floria Aemilia, a concubine to S..."
1753,9780380710829,038071082X,The Grass Crown,Colleen McCullough,Fiction,http://books.google.com/books/content?id=Kt3gN...,The lives of ancient Rome's men--general Gaius...,1992.0,4.29,1104.0,9397.0,The Grass Crown,9780380710829 The lives of ancient Rome's men-...
1754,9780380710843,0380710846,Caesar's Women,Colleen McCullough,Fiction,http://books.google.com/books/content?id=yFK8F...,The fourth novel of the Masters of Rome series...,1997.0,4.24,943.0,5277.0,Caesar's Women,9780380710843 The fourth novel of the Masters ...
2408,9780451454249,0451454243,The Forest House,Marion Zimmer Bradley,Fiction,http://books.google.com/books/content?id=68RD0...,"In a tale set in Druidic England, Eilan feels ...",1995.0,3.86,462.0,14898.0,The Forest House,9780451454249 In a tale set in Druidic England...
2975,9780671024208,0671024205,The October Horse,Colleen McCullough,Fiction,http://books.google.com/books/content?id=XWWDr...,Now in paperback--the sweeping epic of ancient...,2003.0,4.28,1110.0,4186.0,The October Horse,9780671024208 Now in paperback--the sweeping e...
3074,9780679724773,067972477X,"I, Claudius",Robert Graves,Fiction,http://books.google.com/books/content?id=cRzDT...,The emperor Claudius tells of his life during ...,1989.0,4.27,468.0,43469.0,"I, Claudius: From the Autobiography of Tiberiu...",9780679724773 The emperor Claudius tells of hi...
3182,9780688093686,068809368X,The first man in Rome,Colleen McCullough,Fiction,http://books.google.com/books/content?id=kMRmP...,A story tracing the creation of Republican Rom...,1990.0,4.10,896.0,294.0,The first man in Rome,9780688093686 A story tracing the creation of ...


In [ ]:
category_mapping = {
    'Fiction' : "Fiction",
    'Juvenile Fiction': "Children's Fiction",
    'Biography & Autobiography': "Nonfiction",
    'History': "Nonfiction",
    'Literary Criticism': "Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction",
    'Comics & Graphic Novels': "Fiction",
    'Drama' : "Fiction",
    'Juvenile Nonfiction': "Children's Nonfiction",
    'Science': "Nonfiction",
    "Poetry": "Fiction"}

books["simple_categories"] = books["categories"].map(category_mapping)

In [ ]:
from transformers import pipeline


book_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device="cuda")


Device set to use cuda


In [28]:
sequence = books.loc[books["categories"] == "Fiction", "description"].reset_index(drop=True)[0]
sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [29]:
pipe(sequence, book_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [30]:
import numpy as np

max_index = np.argmax(pipe(sequence, book_categories)["scores"])
max_label = pipe(sequence, book_categories)['labels'][max_index]
max_label

'Fiction'

In [31]:
def generate_predictions(sequence: str, categories: list) -> str:
    """
    Generate the most likely category prediction for a given text sequence.

    Args:
        sequence (str): The text sequence to classify.
        categories (list): A list of possible categories for classification.

    Returns:
        str: The category with the highest predicted score.
    """
    # Use the zero-shot classification pipeline to get predictions
    predictions = pipe(sequence, categories)
    
    # Find the index of the highest score
    max_index = np.argmax(predictions["scores"])
    
    # Retrieve the label corresponding to the highest score
    max_label = predictions["labels"][max_index]
    
    return max_label

In [38]:
from tqdm import tqdm

# Initialize lists to store actual and predicted categories
actual_cats = []
predicted_cats = []

# Iterate over the first 300 descriptions of books in the "Fiction" category
# Generate the predicted category for the description using the zero-shot classification pipeline
for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, book_categories)]
    actual_cats += ["Fiction"]

100%|██████████| 300/300 [01:48<00:00,  2.76it/s]


In [39]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, book_categories)]
    actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [02:54<00:00,  1.72it/s]


In [40]:
predictions_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})

In [41]:
predictions_df

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Nonfiction
4,Fiction,Fiction
...,...,...
595,Nonfiction,Nonfiction
596,Nonfiction,Fiction
597,Nonfiction,Nonfiction
598,Nonfiction,Nonfiction


In [43]:
predictions_df["correct_prediction"] = np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)

In [44]:
predictions_df["correct_prediction"].sum() / len(predictions_df)

0.7783333333333333

In [ ]:
# predict the categories for the rows with missing simple categories
isbns = []
predicted_cats = []

missing_cats = books.loc[books["simple_categories"].isna(),["isbn13", "description"]].reset_index(drop=True)

In [46]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats += [generate_predictions(sequence, book_categories)]
    isbns += [missing_cats["isbn13"][i]] 


100%|██████████| 1454/1454 [11:18<00:00,  2.14it/s]


In [47]:
missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})

In [48]:
missing_predicted_df

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1449,9788125026600,Nonfiction
1450,9788171565641,Fiction
1451,9788172235222,Fiction
1452,9788173031014,Nonfiction


In [49]:
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")
books["simple_categories"] = np.where(books["simple_categories"].isna(), books["predicted_categories"], books["simple_categories"])
books = books.drop(columns = ["predicted_categories"])

In [50]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [51]:
books.to_csv("books_with_categories.csv", index=False)